In [0]:
import requests
from databricks.sdk import WorkspaceClient

In [0]:
parent_path_suffix = dbutils.widgets.get("Parent_path_suffix")
warehouse_id = dbutils.widgets.get("Warehouse_id")

In [0]:
workspace_client = WorkspaceClient()
host = workspace_client.config.host
print(host)

In [0]:
user_name = workspace_client.current_user.me().user_name
print(user_name)


In [0]:
headers = {'Authorization' : 'Bearer {}'.format(dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get())}

In [0]:
catalog = "dkushari_uc"
schema = "starbucks_genie"
volume = "exp_imp"
file_name = "serialized_space.json"
volume_file_path = f"/Volumes/{catalog}/{schema}/{volume}/{file_name}"

In [0]:
resp = workspace_client.files.download(volume_file_path)
serialized_space = resp.contents.read().decode("utf-8")
print(serialized_space)

In [0]:
genie_space_description = """This Genie space can answer questions about the business related to a wholesale supplier. It uses data from a wholesale supplier business of a large company that buys parts from suppliers, manages inventory, and sells to customers. Wholesale Distribution: The company purchases parts from suppliers, manages them in inventory, and sells them to customers through orders. Customers and Orders: Customers place orders; each order can have multiple line items (products/parts). Suppliers and Parts: Parts are provided by suppliers and have costs, available quantities, and supply dates. Geography: Nations and regions provide a way to model international operations and shipping. Pricing and Discounts: The data captures extended prices, discounts, and taxes per line item. Shipments: Each line item has ship dates, commit dates, and receipt dates to simulate logistics.
"""
genie_space_title="Imported_Wholesale_Supplier"

In [0]:
payload = {
    "warehouse_id": f"{warehouse_id}",
    "parent_path": f"/Workspace/Users/{user_name}/{parent_path_suffix}",
    "serialized_space": serialized_space,
    "title": genie_space_title,
    "description": genie_space_description
}

In [0]:
resp = requests.post(f"{host}/api/2.0/genie/spaces", json=payload, headers=headers)
print(resp.json())

In [0]:
imported_genie_space_id = resp.json()["space_id"]
print(imported_genie_space_id)


In [0]:
imported_genie_space_description = f"This is an imported Genie space using Genie Export and Import feature. {genie_space_description}"
print(imported_genie_space_description)


In [0]:
warehouse_id_imp = "383cc3b75ffe2072"

In [0]:
payload_imp = {
    "warehouse_id": warehouse_id_imp,
    "description": imported_genie_space_description
}

In [0]:
resp_imp = requests.patch(f"{host}/api/2.0/genie/spaces/{imported_genie_space_id}", json=payload_imp, headers=headers)
print(resp_imp.json())
